In [24]:
import os
import json
import pandas as pd
import requests
from dotenv import load_dotenv

In [25]:
year = 2023  # adjust as needed
folder_path = f"../../data/ncaamb/api_data/match_stats/{year}"

# List all JSON files in the folder
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
print(f"📂 Found {len(json_files)} game files")


📂 Found 5826 game files


In [26]:
rows = []

for filename in json_files:
    file_path = os.path.join(folder_path, filename)

    try:
        with open(file_path, "r") as f:
            game_data = json.load(f)
        if game_data.get("status")!="closed":
            continue
        # Loop for home & away
        for side in ["home", "away"]:
            team = game_data.get(side, {})
            if not team:
                continue
            # print("-----------")
            common_info = {
                # game details
                "game_id": game_data.get("id"),
                "game_sr_id": game_data.get("sr_id"),
                "scheduled": game_data.get("scheduled"),
                "duration": game_data.get("duration"),
                "game_date":game_data.get("scheduled").split('T')[0],
                "status": game_data.get("status"),
                "attendance":game_data.get("attendance"),
                "track_on_court":game_data.get("track_on_court", "not preasent"),
                # team details
                "team_id": team.get("id"),
                "team_sr_id": team.get("sr_id"),
                "team_name": team.get("name"),
                "team_alias": team.get("alias"),
                "team_market": team.get("market"),
                "points": team.get("points"),
                "bonus": team.get("bonus"),
                "timeouts_remaining": team.get("remaining_timeouts"),
                # team records
                # "record_wins":team.get("record").get("wins"),
                # "record_losses":team.get("record").get("losses"),
                
            }
            
            record = team.get("record", {})
            common_info.update({
                     "record_wins":record.get("wins"),
                     "record_losses":record.get("losses")
                })
            # venue details
            venue = game_data.get("venue", {})
            common_info.update({
                "venue_id": venue.get("id"),
                "venue_name": venue.get("name"),
                "venue_capacity": venue.get("capacity"),
                "venue_address": venue.get("address"),
                "venue_city": venue.get("city"),
                "venue_state": venue.get("state"),
                "venue_zip": venue.get("zip"),
                "venue_country": venue.get("country"),
                "venue_sr_id": venue.get("sr_id"),
            })

            # location details (if present inside venue)
            location = venue.get("location", {})
            common_info.update({
                "venue_lat": location.get("lat"),
                "venue_lon": location.get("lng"),
            })
            
            # player statistics
            player_stats_data = team.get("players", [])
            if player_stats_data:  # list of players
                for player in player_stats_data:
                    # basic info (excluding statistics key)
                    row = common_info.copy()
                    for k, v in player.items():
                        if k != "statistics":
                            row[f"player_{k}"] = v

                    # player statistics
                    player_statistics = player.get("statistics", {})
                    for stat_name, stat_value in player_statistics.items():
                        row[f"{stat_name}"] = stat_value

                    # player periodic statistics
                    periods = player_statistics.get("periods", [])
                    for p in periods:
                        period_no = p.get("number")
                        suffix = f"{period_no}th_period"
                        for stat_name, value in p.items():
                            if stat_name != "number":
                                row[f"{suffix}_{stat_name}"] = value
                    rows.append(row)
                
            

    except Exception as e:
        print(f"❌ Failed {filename}: {e}")
        break


In [27]:
df = pd.DataFrame(rows)
df.drop(columns=['periods'],inplace=True)
# Composite key = (game_id, team_id)
print(f"✅ Processed {len(df)} rows from {len(json_files)} files")

output_path = f"../../data/ncaamb/api_data/player_stats/ncaamb_player_stats_{year}.csv"
df.to_csv(output_path, index=False)
print(f"📄 Saved CSV → {output_path}")


✅ Processed 183468 rows from 5826 files
📄 Saved CSV → ../../data/ncaamb/api_data/player_stats/ncaamb_player_stats_2023.csv


### Analyze the distribution of collected data

In [28]:
# df = pd.read_csv('../../data/ncaamb/api_data/')
df = pd.read_csv(output_path)
df.shape

C:\Users\ManabodhaMahananda\AppData\Local\Temp\ipykernel_11508\2862003217.py:2: DtypeWarning: Columns (3,24,27,28,35,171,172,174,215) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_path)


(183468, 216)

In [ ]:
# Number of player of a team
unique_players_per_game = df.groupby('game_id')['player_id'].nunique()
unique_players_dict = unique_players_per_game.to_dict()
print(unique_players_per_game)
print(unique_players_dict)


game_id
0005de91-92e8-4a20-b2a3-843dce2f8057    35
000ca765-aa1e-4b06-bf6b-354faf07d835    31
0010d715-b84e-47d4-a990-084a5eaff6e8    28
0015deef-7fff-4d90-9aed-5b2705c366bb    35
001aa366-5640-4aa1-baab-fd358470624d    33
                                        ..
fff2dd6f-3f66-4238-97c3-c1a66b2306ee    29
fff3e80d-2e12-420d-9e8b-5f426f8791b5    30
fff420f2-1984-4d49-9019-823e03710627    28
fff65681-ba14-487c-80c2-8141ab1f484d    33
fffcacff-7b52-495c-ac83-88c214049a9c    30
Name: player_id, Length: 5821, dtype: int64
{'0005de91-92e8-4a20-b2a3-843dce2f8057': 35, '000ca765-aa1e-4b06-bf6b-354faf07d835': 31, '0010d715-b84e-47d4-a990-084a5eaff6e8': 28, '0015deef-7fff-4d90-9aed-5b2705c366bb': 35, '001aa366-5640-4aa1-baab-fd358470624d': 33, '001e641a-968a-44d4-bda6-b30e28f1ae40': 34, '002af340-03fd-4719-8162-9d27138f6203': 30, '002e935d-6141-4012-924a-dc5da0756f83': 29, '003746d1-5006-4445-a299-fc226d1ed8a2': 33, '003d5364-5242-4655-9880-c326501f465b': 28, '003fe374-98af-4334-b963-d65575df9

In [ ]:
# Check for duplicate game_id + player_id pairs
duplicate_pairs = df.duplicated(subset=['game_id', 'player_id'], keep=False)
if duplicate_pairs.any():
    print("There are duplicate game_id + player_id pairs:")
    print(df[duplicate_pairs])
else:
    print("All game_id + player_id pairs are unique.")


All game_id + player_id pairs are unique.


In [ ]:
# If each game id have unique 2 team id
games_per_team = df.groupby('game_id')['team_id'].nunique()
print(games_per_team)
invalid_games = games_per_team[games_per_team != 2]
if not invalid_games.empty:
    print("Games with teams other than 2:")
    print(invalid_games)
else:
    print("All games have exactly 2 teams.")


game_id
0005de91-92e8-4a20-b2a3-843dce2f8057    2
000ca765-aa1e-4b06-bf6b-354faf07d835    2
0010d715-b84e-47d4-a990-084a5eaff6e8    2
0015deef-7fff-4d90-9aed-5b2705c366bb    2
001aa366-5640-4aa1-baab-fd358470624d    2
                                       ..
fff2dd6f-3f66-4238-97c3-c1a66b2306ee    2
fff3e80d-2e12-420d-9e8b-5f426f8791b5    2
fff420f2-1984-4d49-9019-823e03710627    2
fff65681-ba14-487c-80c2-8141ab1f484d    2
fffcacff-7b52-495c-ac83-88c214049a9c    2
Name: team_id, Length: 5821, dtype: int64
All games have exactly 2 teams.


In [ ]:
# Total number of team, games, players in each team
games_per_team = df.groupby('team_id')['game_id'].nunique()
players_per_team = df.groupby('team_id')['player_id'].nunique()
team_summary = pd.DataFrame({
    'total_games': games_per_team,
    'total_players': players_per_team
}).reset_index()

print(team_summary)


                                  team_id  total_games  total_players
0    006081fb-2387-49d3-8578-f7b873ca0795            1             22
1    0085b9b7-7d84-43bf-8dc2-95f698f4ae23            1             17
2    0095032d-6143-44f2-8974-f6815fc56c5b           31             17
3    0113eea0-c943-4fff-9780-ae0fb099e7ef           30             15
4    017b5377-f359-44f0-a7a6-7c9f067af9c9            1             20
..                                    ...          ...            ...
713  fe406882-9f22-495e-9df6-ef357a6803c6           27             15
714  fea46ac5-6dad-43cd-a770-75554dbcc118           31             18
715  ff34c6ba-fc53-4fa2-87ad-4d3ba5f6cbdc            3             16
716  ff696a32-55b6-4508-997f-0838fb421d4b            1             21
717  ffa04cde-aa2d-4e91-b2cd-bde2bfed44de           30             15

[718 rows x 3 columns]


In [32]:
# Frequency of each unique number of games per team
games_per_team = df.groupby('team_id')['game_id'].nunique()
unique_games_freq = games_per_team.value_counts().sort_index()
print("Frequency of each unique number of games per team:")
print(unique_games_freq)

Frequency of each unique number of games per team:
game_id
1     250
2      78
3      15
4       9
6       1
8       3
27      4
28      6
29     23
30     28
31    290
32     11
Name: count, dtype: int64
